In [1]:
pip install transformers datasets evaluate accelerate

In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [61]:
from transformers import pipeline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

# **Датасет lenta.ru 2019-2023**

In [12]:
df = pd.read_csv('lenta_ru_news_2019_2023.csv'
#                  , sep=',', on_bad_lines='skip'
                )

In [13]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2019/12/15/prsm/,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия,Общество,2019-12-15
1,https://lenta.ru/news/2019/12/15/fb/,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия,Политика,2019-12-15
2,https://lenta.ru/news/2019/12/15/kino/,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура,Фотография,2019-12-15
3,https://lenta.ru/news/2019/12/15/alba/,Полицейские застрелили порезавшего мать буйног...,В Москве полицейские застрелили мужчину при по...,Силовые структуры,Криминал,2019-12-15
4,https://lenta.ru/news/2019/12/15/anons/,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир,Политика,2019-12-15


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496257 entries, 0 to 496256
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     496257 non-null  object
 1   title   496255 non-null  object
 2   text    496257 non-null  object
 3   topic   496173 non-null  object
 4   tags    491110 non-null  object
 5   date    496257 non-null  object
dtypes: object(6)
memory usage: 22.7+ MB


In [15]:
df = df.dropna()
df['topic'] = df['topic'].astype('category')

In [16]:
df.value_counts('tags').head(30)

tags
Политика                97275
Общество                55367
Украина                 33421
Происшествия            21214
Следствие и суд         18994
Госэкономика            17442
Дом                     11794
Футбол                   9567
Рынки                    9126
Бизнес                   8444
Люди                     8307
ТВ и радио               7608
Мир                      7563
Кино                     7491
Оружие                   7465
Музыка                   7453
Белоруссия               7053
Явления                  6617
Интернет                 6600
Россия                   5703
Бокс и ММА               5673
Наука                    5414
Полиция и спецслужбы     5075
Криминал                 4838
Закавказье               4503
Внешний вид              4405
Город                    4319
Стиль                    4067
Зимние виды              3959
Все                      3857
Name: count, dtype: int64

In [17]:
tags_to_keep = ['Политика', 'Госэкономика', 'Происшествия', 'Бизнес', 'Кино', 'Музыка', 'Внешний вид', 'Город', 'Стиль', 'Рынки', 'Футбол', 'Зимние виды']

filtered_df = df[df['tags'].isin(tags_to_keep)]

In [18]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 194762 entries, 1 to 496256
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   url     194762 non-null  object  
 1   title   194762 non-null  object  
 2   text    194762 non-null  object  
 3   topic   194762 non-null  category
 4   tags    194762 non-null  object  
 5   date    194762 non-null  object  
dtypes: category(1), object(5)
memory usage: 9.1+ MB


In [19]:
filtered_df['is_politics'] = filtered_df['tags'].apply(lambda x: 1 if x == 'Политика' else 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491110 entries, 0 to 496256
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   url     491110 non-null  object  
 1   title   491110 non-null  object  
 2   text    491110 non-null  object  
 3   topic   491110 non-null  category
 4   tags    491110 non-null  object  
 5   date    491110 non-null  object  
dtypes: category(1), object(5)
memory usage: 23.0+ MB


C:\Users\User\AppData\Local\Temp\ipykernel_3216\3168639129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['is_politics'] = filtered_df['tags'].apply(lambda x: 1 if x == 'Политика' else 0)


In [20]:
# Случайным образом отбираем 50000 строк
sampled_df = filtered_df.sample(n=50000, random_state=42)

In [21]:
# Делим данные на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(
    sampled_df['text'], sampled_df['is_politics'], 
    test_size=0.25, random_state=42,stratify=sampled_df['is_politics']
)

# **Пример с хаггинфэйс на претренированном датасете**

In [33]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

sequence_to_classify = "Тейлор Свифт поп и кантри певица"
candidate_labels = ["politics", "economy", "entertainment", "environment"]

output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

C:\Users\User\anaconda3\envs\pytorch-gpu-test\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'sequence': 'Тейлор Свифт поп и кантри певица', 'labels': ['entertainment', 'environment', 'economy', 'politics'], 'scores': [0.8593404293060303, 0.05369052290916443, 0.04956693574786186, 0.03740214183926582]}


In [51]:
pip install dask

In [52]:
import dask.dataframe as dd

def classify_batch(batch):
    candidate_labels = ["politics", "economy", "entertainment", "environment"]
    return batch.apply(lambda text: 1 if classifier(text, candidate_labels, multi_label=False)['labels'][0] == 'politics' else 0)

def classifier_hf(series):
    dask_series = dd.from_pandas(series, npartitions=10)
    results = dask_series.map_partitions(classify_batch).compute()
    return pd.Series(results, index=series.index)

In [57]:
sample = sampled_df.sample(n=1000, random_state=42)

In [58]:
classified_data = classifier_hf(sample['text'])

In [59]:
f1 = f1_score(classified_data, sample['is_politics'])

In [60]:
print(f1)

0.7946768060836502


In [62]:
classification_report(classified_data, sample['is_politics'])

'              precision    recall  f1-score   support\n\n           0       0.75      0.80      0.77       459\n           1       0.82      0.77      0.79       541\n\n    accuracy                           0.78      1000\n   macro avg       0.78      0.79      0.78      1000\nweighted avg       0.79      0.78      0.78      1000\n'